## Final Project Submission

Please fill out:
* Student name: NANCY CHELANGAT
* Student pace: full time
* Scheduled project review date/time: N/A
* Instructor name: Nikita
* Blog post URL: N/A


# Film Studio Recommendation for Microsoft

**Author:** Nancy Chelangat
***

## Background and Overview
***

Newly created Microsoft Film Studios is looking for recommendations on what their first film should be. Analysis of film metadata from IMDB and financial data from The Numbers will be used to make recommendations to the studio.

## Problem Statement
***

Microsoft, one of the big five tech companies in the world, is the last to make the foray into original video content production. While the company may know almost everything about making software people will use, they don't know much about making films people will watch. What they lack in knowledge they make up for in resources, and now they want to throw their weight against the box office and make a successful movie.


Questions being considered:

- What genre of films has been most financially successful?
- When should the film be released?
- How much should the budget be?
- Are there any directors that stand out in that genre?
- Are any particular actors within that genre more successful?

## Data Understanding
***

IMDB, the Internet Movie Database, is one of the largest sites for movie news and information. They provide data such as ratings, film titles, genres, crew and principals, and biographical data. The data files provided for the project are from 2019.

The Numbers is a website for movie financial information. They provide data on production costs and gross sales. The data files provided for the project are also from 2019.

# Set up DataFrames

## Loading datasets

Perform standard imports of pandas, numpy, seaborn, and matplotlib, along with the magic line to print plots in our jupyter notebook.

The 3 main datasets for analysis are:
- title.basics.csv
- title.ratings.csv
- tn.movie_budgets.csv

The first two datasets contain movie name and rating information and will be merged on unique identifier 'tconst', as well as unique IDs to get cast and crew names from the additional datasets. The tn.movie_budgets dataset does not share this identifier, but it has the financial data and release information needed for analysis.

The 3 additional datasets to explore:
- name.basics.csv
- title.crew.csv
- title.principals.csv

The crew and principals datasets contain lists of people who work on or in films, and each entry contains the 'tconst' unique identifiers to link the movie data. They also contain another shared identifier connecting the name.basics dataset for biographical information like names, birth/death dates, and roles.

In [41]:
# import the required libraries 
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib as pyplot
import sqlite3



In [46]:
import zipfile
with zipfile.ZipFile('zippedData/im.db.zip', 'r') as zip_ref:
    zip_ref.extractall('zippedData')

PermissionError: [Errno 13] Permission denied: 'zippedData\\im.db'

In [ ]:
import sqlite3 
conn = sqlite3.connect("zippedData/im.db/im.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_names = cursor.fetchall()
for table_name in table_names:
    print(table_name[0])

cursor.execute("SELECT * FROM movie_ratings;")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

In [ ]:
conn = sqlite3.connect("zippedData/im.db/im.db")
query = """ SELECT * FROM  sqlite_master """
tables = pd.read_sql(query, conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


## Checking for Null Values 

In [ ]:
mr = pd.read_sql("""
SELECT *
FROM movie_ratings;
""", conn)

print(mr, conn)
mb = mr = pd.read_sql("""
SELECT *
FROM movie_basics;
""", conn)

print(mb, conn)

         movie_id  averagerating  numvotes
0      tt10356526            8.3        31
1      tt10384606            8.9       559
2       tt1042974            6.4        20
3       tt1043726            4.2     50352
4       tt1060240            6.5        21
...           ...            ...       ...
73851   tt9805820            8.1        25
73852   tt9844256            7.5        24
73853   tt9851050            4.7        14
73854   tt9886934            7.0         5
73855   tt9894098            6.3       128

[73856 rows x 3 columns] <sqlite3.Connection object at 0x0000021948AA88B0>
         movie_id                                primary_title  \
0       tt0063540                                    Sunghursh   
1       tt0066787              One Day Before the Rainy Season   
2       tt0069049                   The Other Side of the Wind   
3       tt0069204                              Sabse Bada Sukh   
4       tt0100275                     The Wandering Soap Opera   
...         

In [ ]:
mr.info()
mb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres  

In [ ]:
print('Total movie_ratings prior to cleaning:', len(mr))
print('movie_ratings null values:')
for col in mr.columns:
    print('- ', col, '-', mr[col].isna().sum())
print('\n')

print('Total movie_basics prior to cleaning:', len(mb))
print('movie_basics null values:')
for col in mb.columns:
    print('- ', col, '-', mb[col].isna().sum())
print('\n')



Total movie_ratings prior to cleaning: 146144
movie_ratings null values:
-  movie_id - 0
-  primary_title - 0
-  original_title - 21
-  start_year - 0
-  runtime_minutes - 31739
-  genres - 5408


Total movie_basics prior to cleaning: 146144
movie_basics null values:
-  movie_id - 0
-  primary_title - 0
-  original_title - 21
-  start_year - 0
-  runtime_minutes - 31739
-  genres - 5408




In [ ]:
pd.read_sql("""SELECT * FROM movie_ratings;""", conn)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


***
Null value decisions:
- Of the 73,856 records, only 804 do not have a genre (approximately 1%). Rows not containing a genre will be dropped
- There are no plans to use runtime_minutes as part of the film analysis, so the column will be dropped

In [ ]:

mr.drop('runtime_minutes', axis=1, inplace=True)
mr.dropna(inplace=True)





KeyError: "['runtime_minutes'] not found in axis"